Evaluation of fetching trades with raw_data or df

In [14]:
import os,sys
# sys.path.append(os.path.dirname(os.path.dirname(os.path.abspath(__file__))))
# print(os.path.dirname(os.path.dirname(os.path.abspath(__file__))))
from alpaca.data.historical import StockHistoricalDataClient
from alpaca.data.requests import CryptoLatestTradeRequest, StockLatestTradeRequest, StockLatestBarRequest, StockTradesRequest
from alpaca.data.enums import DataFeed
from v2realbot.config import ACCOUNT1_PAPER_API_KEY, ACCOUNT1_PAPER_SECRET_KEY, ACCOUNT1_LIVE_API_KEY, ACCOUNT1_LIVE_SECRET_KEY
from v2realbot.utils.utils import zoneNY, zoneUTC
from v2realbot.loader.aggregator_vectorized import convert_dict_to_multiindex_df
from datetime import datetime, timezone, time, timedelta, date
from rich import print
import time
import pytz

parametry = {}
symbol = ["BAC"]
client = StockHistoricalDataClient(ACCOUNT1_LIVE_API_KEY, ACCOUNT1_LIVE_SECRET_KEY, raw_data=True)
datetime_object_from = zoneNY.localize(datetime(2024, 10, 25, 9, 31, 8))
datetime_object_to = zoneNY.localize(datetime(2024, 10, 25, 9, 32, 10))

#extract dates
from_date = datetime_object_from.date()
to_date = from_date + timedelta(days=1)
print(from_date, to_date)

trades_request = StockTradesRequest(symbol_or_symbols=symbol, feed = DataFeed.SIP, start=from_date, end=to_date)

#time this for performance
start_time = time.time()
all_trades = client.get_stock_trades(trades_request)
# End the timer
end_time = time.time()

# Calculate elapsed time
elapsed_time = end_time - start_time
print(f"Call duration: {elapsed_time:.4f} seconds")
#print(all_trades)
df = convert_dict_to_multiindex_df(all_trades, rename_labels=False)
df
# df = pd.DataFrame(symbol_data)
# df['t'] = pd.to_datetime(df['t'], utc=True, format="ISO8601") 

#Raw = True, 1 DAY
#35s s 100.000 page_limit #alpaca-py 0.18.1 - DAY
#35s      10.000 limit
#installed 0.31.0

#Raw=False, 1 DAY
#over 3m - unsusable
#dict to df 

#Raw=True, converted explcitily
#39seconds


2024-10-25 2024-10-26

Call duration: 24.6472 seconds

x       p    s               i  \
symbol t                                                                  
BAC    2024-10-25 04:00:03.444353-04:00  K  42.630    2  52983525027902   
       2024-10-25 04:00:21.682664-04:00  K  42.630    8  52983525027906   
       2024-10-25 04:00:21.682959-04:00  K  42.630    1  52983525027907   
       2024-10-25 04:00:21.683004-04:00  K  42.630    1  52983525027908   
       2024-10-25 04:01:14.223302-04:00  P  42.650    1  52983525029453   
...                                     ..     ...  ...             ...   
       2024-10-25 19:58:37.287617-04:00  D  41.880    2  71714347169074   
       2024-10-25 19:58:39.317704-04:00  D  41.890  388  71714347169096   
       2024-10-25 19:58:53.149621-04:00  Z  41.880  209  52983526159987   
       2024-10-25 19:59:53.380330-04:00  D  41.880   11  71714347175754   
       2024-10-25 19:59:53.523889-04:00  D  41.875    1  71714347175755   

                                                    c  z  
symbol t                                                  
BAC    2024-10-25 04:00:03.444353-04:00  [ , F, T, I]  A  
       2024-10-25 04:00:21.682664-04:00     [ , T, I]  A  
       2024-10-25 04:00:21.682959-04:00  [ , F, T, I]  A  
       2024-10-25 04:00:21.683004-04:00  [ , F, T, I]  A  
       2024-10-25 04:01:14.223302-04:00  [ , F, T, I]  A  
...                                               ... ..  
       2024-10-25 19:58:37.287617-04:00     [ , T, I]  A  
       2024-10-25 19:58:39.317704-04:00        [ , T]  A  
       2024-10-25 19:58:53.149621-04:00        [ , T]  A  
       2024-10-25 19:59:53.380330-04:00     [ , T, I]  A  
       2024-10-25 19:59:53.523889-04:00     [ , T, I]  A  

[120947 rows x 6 columns]

In [13]:
df_reset = df.reset_index() # Reset index to remove MultiIndex levels, making them columns
df_reset = df_reset.drop(columns=['symbol']) #remove symbol column
df_reset = df_reset.set_index('timestamp') #reindex by timestamp
df_reset

,exchange,price,size,id,conditions,tape
timestamp,,,,,,
2023-04-12 16:41:17.121739-04:00,D,28.49,16,71697236571957,"[ , T, I]",A
2023-04-12 16:41:29.424150-04:00,P,28.49,60,52983560948816,"[ , T, I]",A
2023-04-12 16:42:11.981869-04:00,D,28.50,10,79372343005240,"[ , T, I]",A
2023-04-12 16:42:12.204190-04:00,D,28.50,3000,79372343005241,"[ , T]",A
2023-04-12 16:42:28.676138-04:00,D,28.50,2000,71697236575044,"[ , T]",A
...,...,...,...,...,...,...
2023-04-13 16:40:56.129792-04:00,D,28.55,55,79372326031955,"[ , T, I]",A
2023-04-13 16:41:00.789216-04:00,K,28.55,300,52983526090587,"[ , F, T]",A
2023-04-13 16:41:00.790254-04:00,P,28.56,11,52983576872501,"[ , F, T, I]",A


In [4]:
# Reset index to remove MultiIndex levels, making them columns
df_reset = df.reset_index()

# Drop the 'symbol' column (assuming it was named 'symbol') that held 'BAC'
df_reset = df_reset.drop(columns=['level_0'])  # Replace 'level_0' with actual name if different

# Alternatively, if 'BAC' is the only value in the symbol column, you can filter it out
# df_reset = df_reset[df_reset['symbol'] != 'BAC']

# Now df_reset is a standard DataFrame without the 'BAC' level in the index

KeyError: 'BAC'

In [12]:
str(datetime_object_from.date())

'2024-10-25'